In [6]:
#Loading environment variables

import os

WEAVIATE_CLUSTER_URL = os.getenv("WEAVIATE_CLUSTER_URL")
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

## Connect to WCS Instance

In [7]:
import weaviate

# Connect to a cloud instance of Weaviate (with WCS)
client = weaviate.connect_to_wcs(
    cluster_url=WEAVIATE_CLUSTER_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_API_KEY),
    headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
        "X-Cohere-Api-Key": COHERE_API_KEY
    },

    additional_config=weaviate.config.AdditionalConfig(timeout=(500,1500))
)

client.is_ready()

/var/folders/3v/_jgc177x2ln06306pq8qlcqc0000gn/T/ipykernel_29222/1302434159.py:4: ResourceWarning: unclosed <ssl.SSLSocket fd=73, family=2, type=1, proto=0, laddr=('100.64.100.6', 55170), raddr=('34.149.137.116', 443)>
  client = weaviate.connect_to_wcs(


True

## RAG 

In [33]:
import cohere
co = cohere.Client(COHERE_API_KEY)

def generate_query_from_prompt(prompt):
    response = co.chat(
        message=prompt,
        search_queries_only=True
    )
    return response.search_queries[0].text

In [34]:
# Example of how to generate a query from a prompt
generate_query_from_prompt("Where do the tallest penguins live?")

'tallest penguins'

In [36]:
from weaviate.classes.query import Filter

def two_step_rag(user_prompt):
    # Step 1
    prompt = user_prompt + " Please only use the provided content with this prompt. Don't make things up."
    
    generated_query = generate_query_from_prompt(prompt)
    print("=== Generated Query ===")
    print(f"Generated query: {generated_query}")

    # Step 2
    wiki = client.collections.get("Wikipedia")

    response = wiki.generate.near_text(
        query=generated_query,
        auto_limit=1,
        grouped_task=prompt,
        grouped_properties=["text", "title"]
    )

    # Print results
    print("=== Generated Response ===")
    print(response.generated)

    print("=== Source ===")
    for item in response.objects:
        print(item.properties)

In [37]:
two_step_rag("How do airplanes fly?")

=== Generated Query ===
Generated query: How do planes fly simple explanation
=== Generated Response ===
The provided content does not contain any information related to how airplanes fly. Therefore, it is not possible to explain how airplanes fly using only the provided content.
=== Source ===
{'text': 'Certain content is copyright Oxford University Press USA. Some phrase translations come from Wikitravel.', 'views': 4305.65283203125, 'wiki_id': 3235536, 'url': 'https://en.wikipedia.org/wiki?curid=3235536', 'title': 'Google Translate', 'lang': 'en', 'lang_id': 0}
{'text': 'Many of the civilians died because of deliberate genocide, massacres, mass bombings, disease, and starvation.', 'views': 4472.27392578125, 'wiki_id': 32927, 'url': 'https://en.wikipedia.org/wiki?curid=32927', 'title': 'World War II', 'lang': 'en', 'lang_id': 0}
{'text': ', the top three countries spreading state-linked Twitter misinformation are Russia, Iran and Saudi Arabia.', 'views': 4185.50732421875, 'wiki_id': 

In [38]:
two_step_rag("What are the pros and cons of automation using computer?")

=== Generated Query ===
Generated query: pros and cons of automation using computer
=== Generated Response ===
Pros of Automation Using Computer:

1. Increased Speed and Efficiency: The rise of the internet has led to near-instant communication through email, instant messaging, and VoIP. This speed and efficiency can be attributed to automation, which allows for faster data transmission over fiber optic networks operating at high speeds.

2. Growth and Expansion: The lack of central administration in the internet allows for organic growth of the network. Automation plays a significant role in this growth, as it enables the efficient handling of increasing amounts of data and information.

3. Vendor Interoperability: The non-proprietary nature of the internet protocols encourages vendor interoperability. Automation using computers can facilitate this interoperability, preventing any one company from exerting too much control over the network.

Cons of Automation Using Computer:

1. Depe

In [40]:
two_step_rag("How do CPUs work?")

=== Generated Query ===
Generated query: how do CPUs work
=== Generated Response ===
The provided content does not contain information on how CPUs (Central Processing Units) work. The content primarily discusses the history and structure of the Internet, Internet Service Providers (ISPs), and data centers. Please provide the relevant content to explain how CPUs work.
=== Source ===
{'text': 'The origins of the Internet date back to the development of packet switching and research commissioned by the United States Department of Defense in the 1960s to enable time-sharing of computers. The primary precursor network, the ARPANET, initially served as a backbone for interconnection of regional academic and military networks in the 1970s to enable resource sharing. The funding of the National Science Foundation Network as a new backbone in the 1980s, as well as private funding for other commercial extensions, led to worldwide participation in the development of new networking technologies, a